# CIFAR10 Competition

CIFAR10 is a public dataset.
It consists of 60000 32x32 colour images in 10 classes, with 6000 images per class (https://www.cs.toronto.edu/~kriz/cifar.html).

The task of this competition is basically to create a model for a classifier that predicts the classes.
This notebook contains a baseline model that you have to beat.
If you are able to beat the baseline, you may submit your model such that you can compete with your classmates.

Let's start with setting up the project.
First, we load the dataset and pre-process it:

In [1]:
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# preprocess data
x_train = ((x_train-128.)/128.).astype(np.float32)
x_test = ((x_test-128.)/128.).astype(np.float32)

y_train = (np.arange(10) == y_train[:]).astype(np.float32)
y_test = (np.arange(10) == y_test[:]).astype(np.float32)

print(x_train[0])
print(y_train[0])

[[[-0.5390625 -0.515625  -0.5078125]
  [-0.6640625 -0.640625  -0.6484375]
  [-0.609375  -0.625     -0.6640625]
  ...
  [ 0.234375   0.03125   -0.15625  ]
  [ 0.1875    -0.0234375 -0.203125 ]
  [ 0.15625   -0.03125   -0.1953125]]

 [[-0.875     -0.84375   -0.84375  ]
  [-1.        -1.        -1.       ]
  [-0.859375  -0.9375    -1.       ]
  ...
  [-0.0390625 -0.3125    -0.5703125]
  [-0.0703125 -0.3515625 -0.609375 ]
  [-0.046875  -0.3203125 -0.5546875]]

 [[-0.8046875 -0.8125    -0.8359375]
  [-0.875     -0.9453125 -1.       ]
  [-0.6171875 -0.7890625 -0.9375   ]
  ...
  [-0.078125  -0.34375   -0.609375 ]
  [-0.0625    -0.34375   -0.609375 ]
  [-0.1484375 -0.4296875 -0.671875 ]]

 ...

 [[ 0.625      0.328125  -0.25     ]
  [ 0.5703125  0.1953125 -0.734375 ]
  [ 0.546875   0.2578125 -0.796875 ]
  ...
  [ 0.25       0.0390625 -0.453125 ]
  [-0.5625    -0.7578125 -0.9453125]
  [-0.5859375 -0.734375  -0.84375  ]]

 [[ 0.40625    0.0859375 -0.25     ]
  [ 0.3515625 -0.0390625 -0.671875 ]


The following function will be used to train the model:

In [2]:
def train(x, y, model, epochs=15, batch_size=1024):
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    correct = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

    y_pred = tf.one_hot(tf.argmax(model, 1), int(model.shape[1]))
    TP = tf.count_nonzero(y_pred * y)
    TN = tf.count_nonzero((y_pred - 1) * (y - 1))
    FP = tf.count_nonzero(y_pred * (y - 1))
    FN = tf.count_nonzero((y_pred - 1) * y)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)
    
    with tf.Session() as sess:
        sess.run(tf.initializers.global_variables())
        #sess.run(tf.local_variables_initializer())
        
        for epoch in range(epochs):
            def run_epoch():
                epoch_loss = 0
                for batch in range(x_train.shape[0]//batch_size):
                    X, Y = x_train[batch*batch_size:(batch+1)*batch_size], y_train[batch*batch_size:(batch+1)*batch_size]
                    _, c = sess.run([optimizer, cost], feed_dict={x:X, y:Y})
                    epoch_loss += c
                print("Epoch %d / %d completed. Loss: %.3f" % (epoch+1, epochs, epoch_loss))
            %time run_epoch()
        
            # compute measures:
            print("accuracy %.4f, precision %.4f, recall %.4f, f1 %.4f"
                  % tuple(sess.run([accuracy, precision, recall, f1], {x:x_test, y:y_test})))

Let us define x and y placeholders:

In [3]:
x = tf.placeholder('float', [None, 32, 32, 3])
y = tf.placeholder('float')

The following function creates the baseline model.
Do not add a softmax layer at the end, since it will be done by the softmax_cross_entropy_with_logits_v2 function later.

In [4]:
def baseline_model(x, classes=10):
    # first reshape the input to a 2d image
    x = tf.reshape(x, shape=[-1, 32, 32, 3])
    
    w = tf.Variable(tf.random_normal([5,5,3,16])) # [filter_height, filter_width, in_channels, out_channels]
    conv1 = tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1,5,5,1], strides=[1,1,1,1], padding='SAME')
    
    w = tf.Variable(tf.random_normal([5,5,16,32])) # [filter_height, filter_width, in_channels, out_channels]
    conv2 = tf.nn.conv2d(conv1, w, strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1,5,5,1], strides=[1,1,1,1], padding='SAME')
    
    w = tf.Variable(tf.random_normal([5,5,32,64])) # [filter_height, filter_width, in_channels, out_channels]
    conv3 = tf.nn.conv2d(conv2, w, strides=[1,1,1,1], padding='SAME')
    conv3 = tf.nn.relu(conv3)
    conv3 = tf.nn.max_pool(conv2, ksize=[1,5,5,1], strides=[1,1,1,1], padding='SAME')
    
    # we need to flatten / reshape the output of the cnn
    cnn_neurons = int(np.prod(conv3.shape[1:]))
    hidden_fc_neurons = 512
    w = tf.Variable(tf.random_normal([cnn_neurons,hidden_fc_neurons]))
    bias = tf.Variable(tf.random_normal([hidden_fc_neurons]))
    fc = tf.reshape(conv3, [-1, cnn_neurons])
    fc = tf.matmul(fc, w)
    fc = fc + bias
    fc = tf.nn.relu(fc)
    
    w = tf.Variable(tf.random_normal([hidden_fc_neurons, classes]))
    bias = tf.Variable(tf.random_normal([classes]))
    output = tf.matmul(fc, w) + bias
    # softmax activation will be done by softmax_cross_entropy_with_logits_v2
    
    return output

In the following cell you can train the baseline model:

In [5]:
model = baseline_model(x)
%time train(x, y, model)

Epoch 1 / 15 completed. Loss: 4609251.324
CPU times: user 16min 53s, sys: 1min 13s, total: 18min 7s
Wall time: 2min 45s
accuracy 0.2793, precision 0.2793, recall 0.2793, f1 0.2793
Epoch 2 / 15 completed. Loss: 1679893.369
CPU times: user 16min 50s, sys: 1min 14s, total: 18min 4s
Wall time: 2min 46s
accuracy 0.3286, precision 0.3286, recall 0.3286, f1 0.3286
Epoch 3 / 15 completed. Loss: 1244279.779
CPU times: user 16min 54s, sys: 1min 14s, total: 18min 8s
Wall time: 2min 44s
accuracy 0.3616, precision 0.3616, recall 0.3616, f1 0.3616
Epoch 4 / 15 completed. Loss: 1033437.229
CPU times: user 16min 55s, sys: 1min 13s, total: 18min 8s
Wall time: 2min 43s
accuracy 0.3867, precision 0.3867, recall 0.3867, f1 0.3867
Epoch 5 / 15 completed. Loss: 889568.027
CPU times: user 16min 58s, sys: 1min 14s, total: 18min 12s
Wall time: 2min 42s
accuracy 0.4010, precision 0.4010, recall 0.4010, f1 0.4010
Epoch 6 / 15 completed. Loss: 779708.188
CPU times: user 16min 56s, sys: 1min 14s, total: 18min 10s


----

## Your Model

Please define in the following cell a function that returns your model.
Replace ```[team_name]``` with an pseudonymous name for your team / submission.
This name will appear in the ranking.
Do not add a softmax layer at the end, since it will be done by the softmax_cross_entropy_with_logits_v2 function later.


In [ ]:
def model_[team_name](x):
    # TODO

In [ ]:
model = model_[team_name](x)
train(x, y, model)

---

## Submission Details

* Submission deadline: November 22nd 2018 11:59 p.m.
* Submission platform: Uniworx (Übungsblatt "CIFAR10 Competition")
* Submission content: Submit only the model definition function (```def model_[team_name](x): ...```) in a text file.

## Rules:

* The model has to be a pure tensorflow model
* You may use any tensorflow operations (convolutions, fully-connected layers, mutiplicatoins, ...)
* Every model will be trained for 15 epochs
* If the training takes more than 8 CPU hours (Intel(R) Core(TM) i7-3770 CPU @ 3.40GHz) your model will be disqualified
* If the model uses more than 16 GB of RAM while training (batch_size=1024), it will be disqualified
* You may build teams or submit by your own. If you submit as team, please submit your model only once.
* You cannot use pre-trained models
* Ranking will be with respect to the F1-score

## Presentation of Results

Results will be presented in the tutorial on November 27th